In [278]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from category_encoders.target_encoder import TargetEncoder
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.metrics import mean_squared_error, r2_score
os.chdir('C:/Users/louka/Desktop/Machine Learning course/Project/Machine-learning-project')
print(os.getcwd())

C:\Users\louka\Desktop\Machine Learning course\Project\Machine-learning-project


In [7]:
#useful functions
def swap_rows(df, row1, row2):
    df.iloc[row1], df.iloc[row2] =  df.iloc[row2].copy(), df.iloc[row1].copy()
    return df
def process_percentage_column(column):
    return column.str.rstrip('%').astype(float) / 100

In [114]:
df_target_2012.dtypes

STATE                     object
Percentage Democrat      float64
Percentage Republican    float64
Percentage Other         float64
dtype: object

In [215]:
#We will get the 2012,2016,2020 datasets
df_2012=pd.read_csv('Data/Features_2012.csv')
df_2016=pd.read_csv('Data/Features_2016.csv')
df_2021=pd.read_csv('Data/Features_2021.csv')

df_target_2012=pd.read_csv('Data/2012 Election Results.csv')
df_target_2016=pd.read_csv('Data/2016 Presidential Election Results by State.csv')
df_target_2020=pd.read_csv('Data/2020 elections made.csv')

df_test_2023=pd.read_csv('Data/Features_2023.csv')

df_target_2012.drop([51],axis=0,inplace=True)
df_target_2012[['Percentage Democrat', 'Percentage Republican', 'Percentage Other']] = df_target_2012[['Percentage Democrat', 'Percentage Republican', 'Percentage Other']].apply(pd.to_numeric, errors='coerce')
df_target_2012[['Percentage Democrat', 'Percentage Republican', 'Percentage Other']]=df_target_2012[['Percentage Democrat', 'Percentage Republican', 'Percentage Other']].div(100) #fixed

#print(df_target_2012) #nice
df_target_2020[['Percentage Democrat', 'Percentage Republican', 'Percentage Other']]=df_target_2020[['Percentage Democrat', 'Percentage Republican', 'Percentage Other']].apply(process_percentage_column)

#Merge feature data
df_train=pd.concat([df_2012,df_2016,df_2021],ignore_index=True)
# print(df_train)

# #Merge target data
df_target=pd.concat([df_target_2012,df_target_2016,df_target_2020],ignore_index=True)
#print(df_target)
#Make indices nice etc
df_train.set_index("NAME",inplace=True) #nice
df_target.set_index("STATE",inplace=True)
df_test_2023.set_index("NAME",inplace=True)
#print(df_train)

#Fixes:
#GRAPI is wrong for anything other than 2012. RIP
df_train.drop(df_train.columns[df_train.columns.str.contains('GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME')], axis=1, inplace=True)
df_test_2023.drop(df_test_2023.columns[df_test_2023.columns.str.contains('GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME')], axis=1, inplace=True)

#df_train

In [203]:
#print(df_train.loc["Alabama"],'\n')
print(df_train['Mean family income'])

NAME
Alabama           68525
Alaska            95924
Arizona           74275
Arkansas          64235
California        91046
                  ...  
Virginia         131733
Washington       135552
West Virginia     87614
Wisconsin        107124
Wyoming          105191
Name: Mean family income, Length: 153, dtype: int64


In [158]:
# df_test.drop(columns=['25+ no highschool'],inplace=True)

In [7]:
# df_train.drop(columns=['25+ no highschool'],inplace=True)

In [231]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
#Set X and y
X = df_train.reset_index(drop=True)
y = df_target.reset_index(drop=True)
print(y.shape)
k= 7 #number of folds. 7 is a good candidate for a balanced R^2 mean.
kf = KFold(n_splits=k, shuffle=True, random_state=8) #make kfold item
R2=[]
y_pred_array=[]
for (train_index, test_index) in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    #print(X_train,X_test)
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    print(X_test.index)
    reg=LinearRegression()
    reg.fit(X_train,y_train)
    y_pred=reg.predict(X_test)
    print('R^2= %.2f' %r2_score(y_test,y_pred))
    R2.append(r2_score(y_test,y_pred))
    y_pred_array.append(y_pred)
    print(y_pred,'\n')
    #confusion_matrix(y_test,y_pred)
R2_mean=np.mean(R2)

#print('The mean R² score is: %.2f' %R2_mean)

(153, 3)
Index([  4,  23,  27,  29,  30,  50,  56,  57,  65,  69,  77,  80,  97, 113,
       123, 129, 131, 134, 138, 140, 145, 147],
      dtype='int64')
R^2= 0.64
[[ 5.08127547e-01  4.58975548e-01  1.29217774e-03]
 [ 5.31003975e-01  4.41713116e-01 -3.60188178e-04]
 [ 4.44047270e-01  5.27025224e-01 -1.21673150e-03]
 [ 5.88744814e-01  3.94327034e-01 -2.77048339e-03]
 [ 6.02738629e-01  3.93918091e-01 -4.50651766e-03]
 [ 3.46031930e-01  6.16898562e-01  2.61243794e-03]
 [ 5.30947877e-01  3.66121565e-01  4.23550072e-02]
 [ 5.62778924e-01  3.71031183e-01  3.95640107e-02]
 [ 3.30861993e-01  5.98659797e-01  3.84944178e-02]
 [ 3.40501828e-01  6.14597639e-01  2.97084655e-02]
 [ 3.82196442e-01  5.37042550e-01  4.39749426e-02]
 [ 4.93657719e-01  4.35985166e-01  3.68617884e-02]
 [ 4.95862393e-01  4.41150169e-01  3.20955118e-02]
 [ 3.95576841e-01  5.73423239e-01  4.22378831e-02]
 [ 6.31608954e-01  3.40028274e-01  2.48976215e-02]
 [ 4.36542722e-01  5.45673356e-01  2.60944473e-02]
 [ 5.25511338e-01  

In [9]:
reg.coef_

array([[-1.29141477e-02, -3.44823102e-02,  1.51805529e-02,
         1.98868674e-02, -4.68553270e-03, -1.67065163e-01,
        -1.63058910e-01, -1.63643923e-01, -1.74174061e-01,
        -1.82345668e-01,  2.55330674e-03, -7.98633504e-02,
        -1.36178922e-01,  7.97663073e-03, -8.95285129e-03,
        -1.74511766e-03,  6.06862978e+11,  6.06862978e+11,
        -1.50161490e-03],
       [ 9.08224192e-04,  4.92664488e-02, -2.78664247e-03,
        -2.04695630e-03,  3.30566788e-02,  1.64338121e-01,
         1.60964964e-01,  1.61562622e-01,  1.70372900e-01,
         1.83561005e-01, -2.58483343e-03,  6.40937795e-02,
         1.29949140e-01, -6.36755282e-03,  9.37365793e-03,
         6.24560332e-03, -6.19549234e+11, -6.19549234e+11,
        -4.09007161e-04],
       [ 1.00359522e-02,  1.38506493e-03, -6.82506475e-04,
        -2.83599017e-03, -6.07526397e-03,  1.10475714e-02,
         1.15137673e-02,  1.05090987e-02,  1.30927996e-02,
         9.43106165e-03,  2.01045376e-04,  1.60744389e-02,
    

In [227]:
#Now, let's perform a prediction on the whole entire sample
y_tot_pred=reg.predict(df_test_2023)
print(y_tot_pred)
df_results=pd.DataFrame(y_tot_pred)
df_results.to_csv(f'Data\Results_2023.csv', index=True)
#R2_tot=r2_score(y_tot_pred,df_test_202)
#print(R2_tot)

[[0.21264675 0.67844643 0.03646196]
 [0.25134295 0.61340769 0.06710336]
 [0.40222985 0.4308889  0.07540428]
 [0.18802696 0.67345147 0.05355125]
 [0.53098073 0.31106072 0.05898909]
 [0.45725074 0.36478568 0.07062693]
 [0.37386461 0.48297406 0.05012829]
 [0.34713721 0.53413345 0.0387369 ]
 [0.66137732 0.2379027  0.04949824]
 [0.37814177 0.44079791 0.06119461]
 [0.32948091 0.54874475 0.04043603]
 [0.45029098 0.42009858 0.04989756]
 [0.21106864 0.63059892 0.06264717]
 [0.39312187 0.48049641 0.0527259 ]
 [0.1951951  0.66914014 0.04920691]
 [0.17988935 0.69094961 0.05033608]
 [0.28393133 0.5741791  0.06470539]
 [0.16656238 0.69722415 0.04700367]
 [0.23771286 0.64562669 0.03745168]
 [0.26418145 0.59509169 0.04436944]
 [0.43730042 0.4383721  0.03708704]
 [0.39453376 0.45697258 0.04858136]
 [0.25350868 0.64264585 0.04109419]
 [0.29523012 0.57749127 0.05037988]
 [0.20675843 0.6981799  0.03167316]
 [0.21226301 0.66182829 0.04870554]
 [0.18708764 0.67706701 0.0542183 ]
 [0.25074675 0.61604438 0.05

In [282]:
#XGBoost Regression
X=df_train
y=df_target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8)
estimators = [
    ("imputation_mean", SimpleImputer(missing_values=np.nan, strategy="mean")),
    ("scaler", StandardScaler()),
    ('clf', XGBRegressor(random_state=8))
]
pipe=Pipeline(steps=estimators)
pipe.fit(X_train,y_train)
y_pred=pipe.predict(X_test)
print(y_pred)
test_r2=r2_score(y_test,y_pred)
print(test_r2)

[[ 5.8764237e-01  4.2208457e-01 -4.4479311e-04]
 [ 6.6405147e-01  2.9864162e-01  5.3207636e-02]
 [ 5.1026726e-01  4.2447793e-01  2.4289617e-02]
 [ 4.2979029e-01  4.6161908e-01  7.7213976e-05]
 [ 6.1258322e-01  4.4474581e-01  2.0710150e-02]
 [ 3.7948176e-01  5.1210403e-01  3.3809431e-02]
 [ 3.4079510e-01  5.5974567e-01  3.3519816e-02]
 [ 4.9558833e-01  4.3185139e-01  4.2652991e-02]
 [ 4.7316393e-01  4.5223954e-01  2.4412265e-02]
 [ 6.0768116e-01  3.9890397e-01  2.3214661e-02]
 [ 5.0349164e-01  4.5410722e-01  1.3812804e-03]
 [ 6.0486197e-01  3.8291955e-01  1.4580834e-02]
 [ 4.6138668e-01  4.9556231e-01  3.5527755e-02]
 [ 4.1797444e-01  5.9514713e-01  4.1693050e-02]
 [ 5.5022734e-01  4.6613246e-01  8.4952917e-05]
 [ 4.7160029e-01  4.4925895e-01  2.6956921e-02]
 [ 3.5453191e-01  5.1257610e-01  4.1193441e-02]
 [ 5.4365855e-01  4.2815158e-01  1.4883623e-02]
 [ 5.6567830e-01  3.8201475e-01  3.3742640e-02]
 [ 4.7854140e-01  3.6013213e-01  4.0924989e-02]
 [ 3.7357980e-01  5.6302679e-01 -4.56653

In [274]:
opt.fit(X_train, y_train)
#print(y_train)
best_model = opt.best_estimator_
y_pred = best_model.predict(X_test)
test_r2 = r2_score(y_test, y_pred)
# Output results
print(y_pred)
print("Best hyperparameters:", opt.best_params_)
print("R^2 score on the test set:", test_r2)

[[0.4525785 0.4693245 0.0540097]
 [0.4525785 0.4693245 0.0540097]
 [0.4525785 0.4693245 0.0540097]
 [0.4525785 0.4693245 0.0540097]
 [0.4525785 0.4693245 0.0540097]
 [0.4525785 0.4693245 0.0540097]
 [0.4525785 0.4693245 0.0540097]
 [0.4525785 0.4693245 0.0540097]
 [0.4525785 0.4693245 0.0540097]
 [0.4525785 0.4693245 0.0540097]
 [0.4525785 0.4693245 0.0540097]
 [0.4525785 0.4693245 0.0540097]
 [0.4525785 0.4693245 0.0540097]
 [0.4525785 0.4693245 0.0540097]
 [0.4525785 0.4693245 0.0540097]
 [0.4525785 0.4693245 0.0540097]
 [0.4525785 0.4693245 0.0540097]
 [0.4525785 0.4693245 0.0540097]
 [0.4525785 0.4693245 0.0540097]
 [0.4525785 0.4693245 0.0540097]
 [0.4525785 0.4693245 0.0540097]
 [0.4525785 0.4693245 0.0540097]
 [0.4525785 0.4693245 0.0540097]
 [0.4525785 0.4693245 0.0540097]
 [0.4525785 0.4693245 0.0540097]
 [0.4525785 0.4693245 0.0540097]
 [0.4525785 0.4693245 0.0540097]
 [0.4525785 0.4693245 0.0540097]
 [0.4525785 0.4693245 0.0540097]
 [0.4525785 0.4693245 0.0540097]
 [0.452578

In [73]:
#XGBoost Classification
#We need to have y as an array and not a matrix
df_train_target['Winner'] = np.where(
    df_train_target['Percentage Democrat'] < df_train_target['Percentage Republican'],
    'Republican',
    'Democrat'
)
df_test_target['Winner']= np.where(
    df_test_target['Percent Democrat'] < df_test_target['Percent Republican'],
    'Republican',
    'Democrat'
)
# print(df_train_target)
# print(df_train_target['Winner'])


In [33]:

X= df_train
y= df_train_target['Winner'].astype("category")
le = LabelEncoder()
le.fit(y)
y=le.transform(y)
X_train, X_test, y_train, y_test = train_test_split(X.reset_index(drop=True), 
    pd.Series(y).reset_index(drop=True), test_size=0.2,shuffle=True, random_state=8)


In [35]:
print(y)

[1 1 1 1 0 0 0 0 0 1 1 0 1 0 1 1 1 1 1 0 0 0 1 0 1 1 1 1 0 1 1 0 0 1 1 1 1
 0 1 0 1 1 1 1 1 0 0 0 1 0 1]


In [57]:

estimators = [
    ('clf', XGBClassifier(tree_method="hist", enable_categorical=True, device="cuda",random_state=8)) # can customize objective function with the objective parameter
]
pipe = Pipeline(steps=estimators)
pipe

Pipeline(steps=[('clf',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device='cuda',
                               early_stopping_rounds=None,
                               enable_categorical=True, eval_metric=None,
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, random_state=8, ...))])

In [67]:

pipe.fit(X_train,y_train)

y_pred = pipe.predict(X_test)
print(confusion_matrix(y_test, y_pred))


y_prob = pipe.predict_proba(X_test)[:, 1]
print(roc_auc_score(y_test, y_prob))

[[20  6]
 [ 2 23]]
0.9500000000000002


In [71]:
X_test= df_test
y_test= df_test_target['Winner'].astype("category")
y_test=le.transform(y_test)
y_pred=pipe.predict(y_test)
y_pred = pipe.predict(X_test)
print(confusion_matrix(y_test, y_pred))


y_prob = pipe.predict_proba(X_test)[:, 1]
print(roc_auc_score(y_test, y_prob))

[[20  6]
 [ 2 23]]
0.9500000000000002


In [39]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
search_space = {
    'clf__max_depth': Integer(2,5),
    'clf__learning_rate': Real(0.001, 1.0, prior='log-uniform'),
    'clf__subsample': Real(0.5, 1.0),
    'clf__colsample_bytree': Real(0.5, 1.0),
    'clf__colsample_bylevel': Real(0.5, 1.0),
    'clf__colsample_bynode' : Real(0.5, 1.0),
    'clf__reg_alpha': Real(0.0, 10.0),
    'clf__reg_lambda': Real(0.0, 10.0),
    'clf__gamma': Real(0.0, 10.0)
}

opt = BayesSearchCV(pipe, search_space, cv=5, n_iter=10, scoring='roc_auc', random_state=8) 
# in reality, you may consider setting cv and n_iter to higher values

In [41]:
y_train.shape

(40,)

In [43]:
opt.fit(X, y)

BayesSearchCV(cv=5,
              estimator=Pipeline(steps=[('encoder', TargetEncoder()),
                                        ('clf',
                                         XGBClassifier(base_score=None,
                                                       booster=None,
                                                       callbacks=None,
                                                       colsample_bylevel=None,
                                                       colsample_bynode=None,
                                                       colsample_bytree=None,
                                                       device='cuda',
                                                       early_stopping_rounds=None,
                                                       enable_categorical=True,
                                                       eval_metric=None,
                                                       feature_types=None,
                                                       gamma=None,
                                                       grow_policy=None,
                                                       importance_type=...
                             'clf__learning_rate': Real(low=0.001, high=1.0, prior='log-uniform', transform='normalize'),
                             'clf__max_depth': Integer(low=2, high=5, prior='uniform', transform='normalize'),
                             'clf__reg_alpha': Real(low=0.0, high=10.0, prior='uniform', transform='normalize'),
                             'clf__reg_lambda': Real(low=0.0, high=10.0, prior='uniform', transform='normalize'),
                             'clf__subsample': Real(low=0.5, high=1.0, prior='uniform', transform='normalize')})

In [44]:
opt.best_estimator_


Pipeline(steps=[('encoder', TargetEncoder(cols=[])),
                ('clf',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=0.5331080642783614,
                               colsample_bynode=0.7262320733948379,
                               colsample_bytree=0.5362052518135637,
                               device='cuda', early_stopping_rounds=None,
                               enable_categorical=True, eval_metric=None,
                               feature_types=None, gamma=7.436578137661654,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None,
                               learning_rate=0.0011002938624638172,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=4, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, random_state=8, ...))])

In [45]:
opt.best_score_

0.5

In [46]:
X_test= df_test
y_test= df_test_target['Winner'].astype("category")
y_test=le.transform(y_test)
y_test

array([1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 1])

In [47]:
opt.score(X_test, y_test)

0.5

In [48]:
opt.predict(X_test)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1])

In [49]:
import xgboost as xgb

#graph = xgb.to_graphviz(opt, num_trees=1)
# Or get a matplotlib axis
#ax = xgb.plot_tree(opt, num_trees=1)
# Get feature importances
opt.best_estimator_.named_steps['clf']

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=0.5331080642783614,
              colsample_bynode=0.7262320733948379,
              colsample_bytree=0.5362052518135637, device='cuda',
              early_stopping_rounds=None, enable_categorical=True,
              eval_metric=None, feature_types=None, gamma=7.436578137661654,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.0011002938624638172,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=8, ...)